# 08 Vector Feature Engineering

**Project:** NORI  
**Author:** Yuseof J  
**Date:** 12/12/25

### **Purpose**
Uses OSM green space layers and NYC Tract layer to calculate the following:
1. Distance to nearest park 
2. Amount of park greenspace within 500m and 1km of tract, respectively

    Note: the tract centroid is used to calculate distances and buffer zones, acting as the tract's representative point.

### **Inputs**
- `data/processed/nyc_tracts.gpkg`
- `data/processed/greenspaces_nyc.gpkg`

### **Outputs**
  
--------------------------------------------------------------------------

### 0. Imports and Setup

In [165]:
# package imports
import os
import pandas as pd
import geopandas as gpd
from pathlib import Path
from shapely.ops import nearest_points
from shapely.geometry import GeometryCollection

# specify filepaths
path_nyc_tracts = 'data/processed/nyc_tracts.gpkg'
path_greenspaces_points = 'data/processed/greenspaces_nyc.gpkg'

# EPSG:2263 - this coordinate reference system is specifically used for high-accuracy mapping of nyc boroughs 
nyc_crs = 'EPSG:2263'

# ensure cwd is project root for file paths to function properly
project_root = Path(os.getcwd())            # get current directory
while not (project_root / "data").exists(): # keep moving up until in parent
    project_root = project_root.parent
os.chdir(project_root)                      # switch to parent directory

### 1. Load Data

In [113]:
# nyc tracts
gdf_nyc_tracts = gpd.read_file(path_nyc_tracts, layer='tracts')

# osm green spaces
gdf_green_nyc_points = gpd.read_file(path_greenspaces_nyc, layer='green_points')
gdf_green_nyc_polygons = gpd.read_file(path_greenspaces_nyc, layer='green_polygons')

### 2. Filter/Format Data

In [114]:
# ensure proper crs
assert(gdf_green_nyc_points.crs.to_string() == nyc_crs)
assert(gdf_green_nyc_polygons.crs.to_string() == nyc_crs)

In [115]:
# filter for parks
gdf_nyc_parks_points = gdf_green_nyc_points[gdf_green_nyc_points.fclass=='park']
gdf_nyc_parks_polygons = gdf_green_nyc_polygons[gdf_green_nyc_polygons.fclass=='park']

#### Point-in-polygon filtering / Deduplication

> Both the point and polygon greenspace layers contain features of class "park". Polygons are ideal for accurate distance-to-park measurements, however points are still useful: even if the park boundaries are not known, points tell us about the existence of a park. I'm choosing to use both points and polygons accordingly. **HOWEVER**, I will be performing point-in-polygon filtering in order to deduplicate any park points that exist within park polygons.

In [116]:
# perform point-in-polygon join
gdf_points_within_polygons = gpd.sjoin(gdf_nyc_parks_points.copy(),
                                       gdf_nyc_parks_polygons[['geometry']].copy(),
                                       how='left',
                                       predicate='within')

# get points within a polygon (for QA purposes)
gdf_duplicate_park_points = gdf_points_within_polygons[gdf_points_within_polygons.index_right.notna()]

print(f"{len(gdf_duplicate_park_points)} / {len(gdf_nyc_parks_points)} points were contained within polygons")

# select only points outside of polygons for distance calculations
gdf_nyc_parks_points_deduplicated = gdf_points_within_polygons[gdf_points_within_polygons.index_right.isna()].copy()
gdf_nyc_parks_points_deduplicated.drop(columns=['index_right'], inplace=True) 

74 / 169 points were contained within polygons


In [120]:
# for visual QA in QGIS
# gdf_duplicate_park_points.to_file("data/processed/duplicate_points.gpkg")

In [118]:
# combine park points and polygons for distance calculations, in this case, combine all polygons and points into one gemetry
# in order to simplify calculation of distance to trct centroids
collection_parks_nyc = GeometryCollection(
    [gdf_nyc_parks_polygons.geometry.union_all()] + list(gdf_nyc_parks_points_deduplicated.geometry)
)

In [119]:
collection_parks_nyc

### 3. Feature Engineering

In [121]:
# get centroid of each tract
gdf_nyc_tracts['centroid'] = gdf_nyc_tracts.geometry.centroid

#### Distance to nearest park

In [122]:
# compute distance between tract centroids and nearest point of polygon in collection_parks_nyc
gdf_nyc_tracts['distance_to_park_m'] = gdf_nyc_tracts.centroid.distance(collection_parks_nyc)

#### Area of nearest park (does the impact of a small nearby park differ from large nearby park?)

> Important note: this calculation only takes into account polygon parks only

In [ ]:
# TODO: include this variable, skipping now for the sake of moving forward with prototyping
# def calculate_nearest_park_area(tract_geom, parks_polygons):    

#### Park area within tract buffer (i.e. how *much* park is accessible)

> This model feature needs clarification. We are not measuring the amount of green space accessible to all tract inhabitants up to 500m/1km outside of tract. Instead, we are measuring the amount of park area within 500m/1km of the tract centroid (representative point). This is for consistency across tracts (i.e. tracts treated the same regardless of shape and size), and simplicity for early modeling. 

In [142]:
# create buffer areas of 500m and 1km for each census tract, wrapping the resulting GeoSeries into a GeoDataFrame
tract_centroid_buffers_500m = gpd.GeoDataFrame(
    gdf_nyc_tracts[['GEOID']],
    geometry=gdf_nyc_tracts.centroid.buffer(500),
    crs=gdf_nyc_tracts.crs
)

tract_centroid_buffers_1km = gpd.GeoDataFrame(
    gdf_nyc_tracts[['GEOID']],
    geometry=gdf_nyc_tracts.centroid.buffer(1000),
    crs=gdf_nyc_tracts.crs
)

In [143]:
# intersect tract buffer zones with nyc parks (creates new polygons)
# here we use only park polygons since park points do not have an area - see model feature documentation for notes
gdf_intersections_500m = gpd.overlay(gdf_nyc_parks_polygons, tract_centroid_buffers_500m, how='intersection')
gdf_intersections_1km = gpd.overlay(gdf_nyc_parks_polygons, tract_centroid_buffers_1km, how='intersection')

In [158]:
# calculate area per polygon (after intersection, each row (polygon) is the part of a park that fell within the buffer zone of a tract- this
# could be either a whole park, or just a part of a park which spanned multiple tract buffer zones)
gdf_intersections_500m['park_area_500m_centroid'] = gdf_intersections_500m.geometry.area
gdf_intersections_1km['park_area_1km_centroid'] = gdf_intersections_1km.geometry.area

# calculate total area of park per tract buffer zone by summing all polygon areas above per GEOID
gdf_park_area_500m = gdf_intersections_500m.groupby('GEOID')['park_area_500m_centroid'].sum()
gdf_park_area_1km = gdf_intersections_1km.groupby('GEOID')['park_area_1km_centroid'].sum()

In [159]:
# join area calculations above to nyc tract gdf on GEOID
gdf_nyc_tracts = gdf_nyc_tracts.merge(gdf_park_area_500m, how='left', on='GEOID')
gdf_nyc_tracts = gdf_nyc_tracts.merge(gdf_park_area_1km, how='left', on='GEOID')

Not all tracts have a value for park_area_... since not all of their centroid buffer zones intersected with a park. These means in gdf_nyc_tracts, they will have no joined value, and therefore a NaN area. Fill this with 0. 

In [163]:
gdf_nyc_tracts[['park_area_500m_centroid', 'park_area_1km_centroid']] = gdf_nyc_tracts[['park_area_500m_centroid', 'park_area_1km_centroid']].fillna(0)

In [164]:
# quick visual check of new columns
gdf_nyc_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,centroid,distance_to_park_m,park_area_500m_centroid,park_area_1km_centroid
0,36,085,024402,36085024402,1400000US36085024402,244.02,Census Tract 244.02,G5020,S,1823028,2065530,+40.4997874,-074.2384712,"MULTIPOLYGON (((912287.455 121583.989, 912302....",POINT (917824.934 121386.913),169.509962,1.206043e+05,6.859291e+05
1,36,085,027705,36085027705,1400000US36085027705,277.05,Census Tract 277.05,G5020,S,531529,0,+40.5882479,-074.1569820,"MULTIPOLYGON (((938978.947 152923.44, 939459.2...",POINT (940648.951 153630.078),2129.879397,0.000000e+00,0.000000e+00
2,36,085,012806,36085012806,1400000US36085012806,128.06,Census Tract 128.06,G5020,S,1319470,580167,+40.5576710,-074.1076715,"MULTIPOLYGON (((950892.186 143465.985, 951155....",POINT (954427.117 142435.083),0.000000,1.370448e+06,4.764153e+06
3,36,047,024400,36047024400,1400000US36047024400,244,Census Tract 244,G5020,S,155278,0,+40.6217475,-073.9862364,"MULTIPOLYGON (((987168.126 166387.055, 987335....",POINT (988070.869 165796.048),622.327717,0.000000e+00,3.497273e+05
4,36,047,023000,36047023000,1400000US36047023000,230,Census Tract 230,G5020,S,150941,0,+40.6378160,-073.9842809,"MULTIPOLYGON (((987443.171 173158.468, 988054....",POINT (988612.664 171650.327),972.144627,0.000000e+00,1.453035e+03


### 4. Save Data

In [167]:
# export processed tract data
perhaps save the feature-enhanced tracts layer to a new layer of the geopackage
gdf_tracts_nyc.to_file(path_output_processed_geodata, layer="tracts")

SyntaxError: invalid syntax (1881589430.py, line 2)